In [85]:
import gensim
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from nltk.tokenize import WordPunctTokenizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# готовая модель word2vec

In [7]:
ru_emb = KeyedVectors.load_word2vec_format("../models/word2vec_embeddings/cc.ru.300.vec")

In [75]:
ru_emb.most_similar('животные')

[('звери', 0.7789862751960754),
 ('Животные', 0.7331661581993103),
 ('млекопитающие', 0.7082071900367737),
 ('насекомые', 0.704063355922699),
 ('питомцы', 0.6828341484069824),
 ('зверушки', 0.6727720499038696),
 ('коты', 0.6652047038078308),
 ('хищники', 0.6650010347366333),
 ('зверьки', 0.6570624113082886),
 ('грызуны', 0.6529560089111328)]

# Токенизация

In [40]:
texts_df = pd.read_excel('../data/raw/unlabeled_texts.xlsx')

In [41]:
texts = list(texts_df['texts'])

In [44]:
tokenizer = WordPunctTokenizer()

In [61]:
data_tok = [tokenizer.tokenize(str(text).lower()) for text in texts]

('Сдаётся студия на длительный срок в доме комфорт класса. В квартире есть всё необходимое для комфортного проживания: кухонный гарнитур, плита, холодильник, чайник, столовый сервиз, столовые приборы, фужеры, диван-кровать, стол, стулья, телевизор, интернет, ванная, стиральная машина. В прихожей имеется удобный шкаф-купе. Всё новое. Дом новый. САО г. Москвы, р-н Дмитровский, ближайшее м. Селигерская и Лианозово. Сейчас там строятся другие станции метро. До метро 10-15 минут. Сдаётся лицам славянской национальности, без животных. Стоимость 42000, стоимость жкх включена. Квартира свободна.',
 ['сдаётся',
  'студия',
  'на',
  'длительный',
  'срок',
  'в',
  'доме',
  'комфорт',
  'класса',
  '.',
  'в',
  'квартире',
  'есть',
  'всё',
  'необходимое',
  'для',
  'комфортного',
  'проживания',
  ':',
  'кухонный',
  'гарнитур',
  ',',
  'плита',
  ',',
  'холодильник',
  ',',
  'чайник',
  ',',
  'столовый',
  'сервиз',
  ',',
  'столовые',
  'приборы',
  ',',
  'фужеры',
  ',',
  'дива

In [ ]:
texts[0], data_tok[0]

# Обучение word2vec на имеющихся текстах

In [177]:
model = Word2Vec(data_tok, 
                 vector_size=300,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

In [178]:
model.most_similar('животные')

[('обсуждаются', 0.9447620511054993),
 ('обговариваются', 0.90764319896698),
 ('возраста', 0.8776581287384033),
 ('дети', 0.8775374293327332),
 ('юр', 0.87453293800354),
 ('индивидуально', 0.8595935702323914),
 ('рассматриваются', 0.8576715588569641),
 ('домашние', 0.8522319793701172),
 ('арендаторы', 0.8483145236968994),
 ('дошкольного', 0.8451724648475647)]

# Топ-слова

In [179]:
words = sorted(model.key_to_index.keys(), 
               key=lambda word: model.get_vecattr(word, "count"),
               reverse=True)[:1000]

In [180]:
words

['.',
 ',',
 'в',
 'и',
 '-',
 'с',
 'на',
 'квартира',
 ':',
 '!',
 'для',
 '(',
 '2',
 'от',
 'метро',
 'есть',
 '"',
 'без',
 'по',
 'м',
 'кухня',
 '3',
 'машина',
 'до',
 'не',
 '/',
 'техникой',
 'все',
 'мебель',
 'рядом',
 'можно',
 'ремонт',
 'срок',
 'техника',
 'стиральная',
 'сдается',
 'длительный',
 'вся',
 'квартире',
 'окна',
 'инфраструктура',
 'доступности',
 '1',
 'гостиная',
 'мебелью',
 'парк',
 'из',
 'предлагается',
 'проживания',
 'магазины',
 'холодильник',
 ').',
 'полностью',
 'дом',
 'во',
 'комнаты',
 'жк',
 'двор',
 'доме',
 'комната',
 'пешком',
 'комиссии',
 'кв',
 'санузел',
 'комнатная',
 'бытовой',
 ')',
 '5',
 'просторная',
 'сдаётся',
 ';',
 'дома',
 'минут',
 'уютная',
 'планировка',
 'шкаф',
 'спальня',
 '10',
 'квартиру',
 'со',
 'территория',
 'гардеробная',
 'интернет',
 'кафе',
 'показ',
 'за',
 'минутах',
 'необходимой',
 'две',
 '+',
 'паркинг',
 'бытовая',
 '),',
 'id',
 'всей',
 'кондиционер',
 'комфортного',
 'или',
 'территории',
 'телев

# Векторы

In [181]:
word_vectors_1 = np.asarray([ru_emb.get_vector(word) for word in words if word in ru_emb.key_to_index.keys()])
word_vectors_2 = np.asarray([model.get_vector(word) for word in words])

In [182]:
word_vectors_1.shape, word_vectors_2.shape

((977, 300), (1000, 300))

# Визуализация

Функция для отображения графиков

In [183]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [184]:
pca = PCA(n_components=2)
scl = StandardScaler()
word_vectors_1_pca = pca.fit_transform(scl.fit_transform(word_vectors_1))
word_vectors_1_pca = scl.fit_transform(word_vectors_1_pca)
draw_vectors(word_vectors_1_pca[:, 0], word_vectors_1_pca[:, 1],
             token=[word for word in words if word in ru_emb.key_to_index.keys()])

Figure(id='3551', ...)

In [185]:
pca.explained_variance_ratio_

array([0.09255072, 0.04612093], dtype=float32)

In [186]:
pca = PCA(n_components=2)
scl = StandardScaler()
word_vectors_2_pca = pca.fit_transform(scl.fit_transform(word_vectors_2))
word_vectors_2_pca = scl.fit_transform(word_vectors_2_pca)
draw_vectors(word_vectors_2_pca[:, 0], word_vectors_2_pca[:, 1],
             token=words)

Figure(id='3815', ...)

In [187]:
model.most_similar('детей')

[('домашних', 0.844732940196991),
 ('маленьких', 0.8337452411651611),
 ('некурящих', 0.8004250526428223),
 ('взрослых', 0.789917528629303),
 ('животных', 0.7783149480819702),
 ('привычек', 0.7774947285652161),
 ('вредных', 0.7638553977012634),
 ('собак', 0.7581284046173096),
 ('аккуратных', 0.7343882918357849),
 ('пары', 0.6927865743637085)]

In [188]:
pca.explained_variance_ratio_

array([0.09787587, 0.07904056], dtype=float32)

# Сравнение моделей

In [189]:
ru_emb.most_similar('детей'), model.most_similar('детей')

([('детишек', 0.8188886642456055),
  ('малышей', 0.8129426836967468),
  ('взрослых', 0.7920259237289429),
  ('деток', 0.7822591662406921),
  ('ребятишек', 0.7797736525535583),
  ('подростков', 0.7711515426635742),
  ('родителей', 0.7524271607398987),
  ('детей-', 0.7370284795761108),
  ('детей.', 0.7123494148254395),
  ('младенцев', 0.7115262150764465)],
 [('домашних', 0.844732940196991),
  ('маленьких', 0.8337452411651611),
  ('некурящих', 0.8004250526428223),
  ('взрослых', 0.789917528629303),
  ('животных', 0.7783149480819702),
  ('привычек', 0.7774947285652161),
  ('вредных', 0.7638553977012634),
  ('собак', 0.7581284046173096),
  ('аккуратных', 0.7343882918357849),
  ('пары', 0.6927865743637085)])

In [190]:
ru_emb.most_similar('курение'), model.most_similar('курение')

([('табакокурение', 0.8035606145858765),
  ('Курение', 0.7048202753067017),
  ('курения', 0.6844199299812317),
  ('курить', 0.6770423650741577),
  ('курении', 0.6505557298660278),
  ('пьянство', 0.6260770559310913),
  ('никотин', 0.625453770160675),
  ('курильщиков', 0.6242698431015015),
  ('курением', 0.6165605187416077),
  ('алкоголь', 0.6049817204475403)],
 [('курить', 0.8438228368759155),
  ('запрещено', 0.8413386344909668),
  ('категорически', 0.7945575714111328),
  ('риелторам', 0.7569586038589478),
  ('нельзя', 0.7517241835594177),
  ('требуется', 0.7325756549835205),
  ('риэлторов', 0.7312472462654114),
  ('платёжеспособные', 0.7263906598091125),
  ('прописан', 0.7261567115783691),
  ('платежеспособные', 0.7249270081520081)])

In [191]:
ru_emb.most_similar('славян'), model.most_similar('славян')

([('русичей', 0.7311936020851135),
  ('Славян', 0.7228178381919861),
  ('германцев', 0.7003255486488342),
  ('славяне', 0.6982201337814331),
  ('славянских', 0.692255437374115),
  ('руссов', 0.6899323463439941),
  ('русов', 0.6740885972976685),
  ('скандинавов', 0.6668840050697327),
  ('индоевропейцев', 0.666354775428772),
  ('славянам', 0.6643184423446655)],
 [('желательно', 0.9089534282684326),
  ('граждан', 0.9054948091506958),
  ('семью', 0.8973556160926819),
  ('строго', 0.8938100337982178),
  ('предпочтение', 0.8842737078666687),
  ('аккуратных', 0.8834877014160156),
  ('арендаторов', 0.8798106908798218),
  ('платежеспособных', 0.8783287405967712),
  ('порядочных', 0.8780922293663025),
  ('русских', 0.8766594529151917)])

In [192]:
ru_emb.most_similar('запрещено'), model.most_similar('запрещено')

([('запрещается', 0.8144304156303406),
  ('разрешено', 0.8113554120063782),
  ('воспрещено', 0.7828918099403381),
  ('запрещалось', 0.761883020401001),
  ('разрешается', 0.7363908886909485),
  ('запрещенно', 0.722277045249939),
  ('воспрещается', 0.7119185924530029),
  ('Запрещено', 0.7018330693244934),
  ('разрешалось', 0.692767858505249),
  ('запрещена', 0.6828192472457886)],
 [('курить', 0.9019497036933899),
  ('нельзя', 0.8819047808647156),
  ('курение', 0.8413386940956116),
  ('прошу', 0.7541925311088562),
  ('категорически', 0.7458579540252686),
  ('!!', 0.724949836730957),
  ('обязательно', 0.7146056294441223),
  ('риэлторов', 0.713988721370697),
  ('пока', 0.7131824493408203),
  ('дети', 0.7124443054199219)])

In [193]:
ru_emb.most_similar('стиральная'), model.most_similar('стиральная')

([('Стиральная', 0.7976304292678833),
  ('посудомоечная', 0.7479637265205383),
  ('стиралка', 0.727093517780304),
  ('стиральную', 0.653666079044342),
  ('стир.', 0.6523314118385315),
  ('сушильная', 0.6423208117485046),
  ('машинка-автомат', 0.6225641965866089),
  ('стиральной', 0.6048418283462524),
  ('микроволновая', 0.5920779705047607),
  ('посудомойка', 0.5899531841278076)],
 [('стир', 0.8832640647888184),
  ('посудомоечная', 0.7869731187820435),
  ('стирально', 0.7283798456192017),
  ('стирки', 0.7217360138893127),
  ('кофе', 0.7018108367919922),
  ('сма', 0.6184381246566772),
  ('сушильная', 0.6029477119445801),
  ('libherr', 0.6016438603401184),
  ('морозильная', 0.6007624268531799),
  ('кондиционер', 0.5992229580879211)])

In [194]:
ru_emb.most_similar('девушки'), model.most_similar('девушки')

([('женщины', 0.802140474319458),
  ('красотки', 0.7774097919464111),
  ('девчонки', 0.7767674922943115),
  ('красавицы', 0.7593241930007935),
  ('парни', 0.7586187124252319),
  ('барышни', 0.7451480627059937),
  ('девочки', 0.7155345678329468),
  ('мужчины', 0.7148702144622803),
  ('дамы', 0.7147289514541626),
  ('девушки.', 0.713891863822937)],
 [('девушек', 0.8983138203620911),
  ('женщин', 0.8376225233078003),
  ('человека', 0.8214417695999146),
  ('муж', 0.8108409643173218),
  ('семейная', 0.8096650838851929),
  ('человеку', 0.8088915348052979),
  ('максимум', 0.8008978962898254),
  ('девушку', 0.7994557023048401),
  ('семья', 0.7961139678955078),
  ('пара', 0.786974310874939)])

In [195]:
ru_emb.most_similar('кавказ'), model.most_similar('кавказ')

([('дагестан', 0.692087709903717),
  ('Кавказ', 0.6875126361846924),
  ('кавказе', 0.6609355211257935),
  ('кавказа', 0.6301323175430298),
  ('крым', 0.618231475353241),
  ('азербайджан', 0.6026282906532288),
  ('алтай', 0.6002475619316101),
  ('донбасс', 0.5705195665359497),
  ('кадыров', 0.5653867125511169),
  ('чечня', 0.5539404153823853)],
 [('помощников', 0.9058140516281128),
  ('валентина', 0.900840163230896),
  ('проведение', 0.8963243365287781),
  ('аккуратность', 0.8884167671203613),
  ('таким', 0.8868986368179321),
  ('заправки', 0.8861851692199707),
  ('!!!!!!!', 0.8857986330986023),
  ('аккуратной', 0.8831799626350403),
  ('ввц', 0.8798341155052185),
  ('труд', 0.8792211413383484)])